In [5]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

import gc
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random
from pathlib import Path
import math

In [2]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    return model
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

In [6]:
tfm = transforms.Compose([
    # transforms.Resize(image_size),
    transforms.ToTensor(),
])

In [12]:
img = Image.open("test.jpg").convert("RGB")
img = tfm(img)
model = get_model_instance_segmentation(2)
model.eval()
 
pred = model(img.unsqueeze(0))
print(pred)

[{'boxes': tensor([[1313.2760,  286.0296, 1758.9568, 1237.1543],
        [ 266.2980,  284.6473,  684.8087, 1221.7095],
        [ 671.6182,  294.9633, 1066.5409, 1201.4760],
        [ 987.9595,  344.5453, 1372.6268, 1221.9905],
        [1667.1614,  508.9835, 1750.5774,  704.6721],
        [1858.6136,  525.4602, 1927.5934,  702.2455],
        [ 230.3424,  343.1954, 1266.8849, 1232.9656],
        [   0.0000, 1125.3748,  295.1183, 1213.9221],
        [ 663.1809,  323.5529, 1047.1290, 1204.9222],
        [ 244.7778,  943.5069,  378.1649, 1201.7089],
        [ 976.0917,  302.5929, 1349.7374, 1243.2539],
        [ 252.3555,  411.7393,  694.8979, 1219.2700],
        [1681.6613,  523.3652, 1804.3450,  710.5125],
        [   0.0000, 1129.5797,  256.3327, 1212.4236],
        [   0.0000, 1126.2727,  256.6265, 1206.2677],
        [ 663.0552,  325.3092, 1049.7162, 1211.4883],
        [1647.5883,  953.1304, 1699.2478, 1145.3979],
        [ 439.1273,  515.6843,  517.7386,  619.7453],
        [  14.111

In [14]:
print(pred[0]['masks'].shape)
# Image.fromarray(pred[0]['masks'][0,0].mul(255).byte().cpu().numpy())

torch.Size([21, 1, 1331, 2000])
